In [382]:
from matrix import *
import numpy as np
import math
import random
from typing import List, Tuple
random.seed(57)

def generate_matrix(rows, cols, min_val, max_val):
    return save_matrix([[random.randint(min_val, max_val) for i in range(cols)] for j in range(rows)])

# Expert 1
Доказать, что оптимальные направления PCA совпадают с собственными векторами матрицы ковариаций. Оптимальные направления означают максимальную дисперсию.

Ковариация: $Cov (X, Y) = (X, Y) = X^T Y$ (скалярное умножение)

Дисперсия: $Dis (X) = ||X||^2, ||X|| = \sqrt(X, X)$

1. Матрица ковариаций: $$\Sigma = \frac{1}{n-1}X X^T,$$ где $X$ - центрированная матрица данных размерности $n \times m$, а $n$ - количество объектов.

    Пусть $W$ - искомые $k$ векторов, $W = (w_1, ..., w_k)$, причём $||w_i|| = 1$ (у них единичные длины). Тогда проекция - это $XW$, и она должна иметь максимальную дисперсию.

    $Dis(XW) = ||XW||^2 = (XW)^T XW = W^TX^T XW$, или $\frac{1}{n-1} W^TX^T XW$ с учётом множителя.

    $Dis(XW) = \frac{1}{n-1} W^TX^T XW = W^T \Sigma W$ - то, что нужно максимизировать.

    Пусть $w_1$ - главная компонента. Найдём $max(w_1^T \Sigma w_1$). Для этого определим $w_2, ..., w_k$, которые так же будут ортогональны $k$ предыдущим ($w_i^T w_j = 0$ при $i \neq j$).

2. $X = USV^T$ - сингулярное разложение, $U$ - матрица левых сингулярных векторов, $V$ - матрица правых сингулярных векторов, $S$ - диагональная матрица сингулярных чисел. $$\Sigma = \frac{1}{n-1} X^TX = \frac{1}{n-1} (USV^T)^T USV = \frac{1}{n-1} VS^T U^T USV =, $$ так как $U$ ортогональна, $$= \frac{1}{n-1} V S^T E SV^T = \frac{1}{n-1} V S^T SV^T$$

    Пусть $\Lambda = \frac{1}{n-1} S^T S$, тогда $\Sigma = V \Lambda V^T$, и это спектральное разложение матрицы.

    $\Lambda$ диагональна, на диагонали собственные значения матрицы, $V$ состоит из собственных векторов.

3. $max(w_1^T \Sigma w_1) \leadsto max(w_1^T (V \Lambda V^T) w_1), ||w_1|| = 1, w_1$ - искомый.

    Замена: $y = V^T w_1$, откуда $w_1 = V \cdot y$. Так как $V$  ортогональна: $||w_1|| = ||Vy|| = ||y|| = 1$.

    Теперь ищем: $max \big[ (Vy)^T (V \Lambda V^T) V y \big]$ при $||y|| = 1$.
    $$(Vy)^T (V \Lambda V^T) Vy = y^T \cdot V^T V \cdot \Lambda \cdot V^T V \cdot y = y^T \cdot E \cdot E \cdot y = y^T  \Lambda y $$

    $$y^T \Lambda y = \begin{pmatrix}
    y_1 & \ldots & y_n
    \end{pmatrix} \begin{pmatrix}
    \lambda_1 & & \\
     & \ddots & \\
     & & \lambda_n
    \end{pmatrix} \begin{pmatrix}
    y_1 \\
    \vdots \\
    y_n
    \end{pmatrix} = \sum^n_{i=1} \lambda_i y_i^2$$

    При этом из условия $||y|| = 1$ следует, что $$\sum^n_{i=1} y^2_i = 1$$

4. Далее надо найти $(\lambda_1 y_1^2 + \lambda_2 y_2^2 + ... + \lambda_n y_n^2)$. Так как $\lambda_1, ..., \lambda_n$ мы сортируем по убыванию, то максимум будет при $y_1 = 1$, а остальные $y_i = 0$.

    $y = (1, 0, ..., 0)^T$, тогда $w_1 = Vy = V \cdot (1, 0, ..., 0)^T = V^{(1)}$ - первый столбец $V$.

    $V^{(1)}$ - в точности собственный вектор, соответствующий максимальному собственному значению, то есть для $w_1$ всё доказано.

5. Теперь приведем рассуждения о выборе следующих компонент ($w_2, ..., w_k$). Нужна ортогональность в $w_1$ и $w_2$:

    Пусть $w_1 = Vy_1$, где $y_1 = (1, 0, ..., 0)^T$, а $w_2 = Vy_2$.

    $$(w_1, w_2) = 0 \Rightarrow (Vy_1, Vy_2) = (Vy_1)^T Vy_2 = y_1^T V^T V y_2 = y_1^T y_2 = 0 $$

    Так как $(1, 0, ..., 0)^T y_2$ = 0, то первая компонента $y_2$ нулевая, остальные любые. Тогда $max(\lambda_1 y_1^2 + \lambda_2 y_2^2 + ... + \lambda_n y_n^2) \leadsto max( \lambda_2 y_2^2 + ... + \lambda_n y_n^2)$, и аналогично получаем $y_2 = (0, 1, 0, ..., 0)^T$, откуда $w_2 = V^{(2)}$ (нашли второй столбец и второй собственный вектор).

Продолжив алгоритм, получим искомые $w_1, w_2, ..., w_k$, равные собственным векторам матрицы $\Sigma$, соответствующие в точности максимальным собственным значениям.

# Easy 1: Метод Гаусса

In [383]:
def gauss_solver(A: Matrix, b: Matrix, tol: float = 1e-5) -> List[Matrix]:
    if A.rows != b.rows or b.columns != 1:
        raise ValueError("Размеры A и b не согласованы")

    n, m = A.rows, A.columns
    # преобразуем в плотный формат
    classic_A = get_classic_format(A)
    classic_b = get_classic_format(b)

    Ab = [row.copy() + [classic_b[i][0]] for i, row in enumerate(classic_A)]

    rank = 0
    # Прямой ход Гаусса
    for col in range(m):
        max_row = rank
        for row in range(rank + 1, n):
            if abs(Ab[row][col]) > abs(Ab[max_row][col]):
                max_row = row

        # Если максимальный элемент равен нулю, скипаем столбец
        if abs(Ab[max_row][col]) < tol:
            continue

        # Меняем строки местами
        Ab[rank], Ab[max_row] = Ab[max_row], Ab[rank]

        # Нормализация строки
        pivot = Ab[rank][col]
        for j in range(col, m + 1):
            Ab[rank][j] /= pivot

        # Исключаем переменную из других строк
        for i in range(n):
            if i != rank and abs(Ab[i][col]) > tol:
                factor = Ab[i][col]
                for j in range(col, m + 1):
                    Ab[i][j] -= factor * Ab[rank][j]

        rank += 1
        if rank == n:
            break

    # Проверяем совместность
    for i in range(rank, n):
        if abs(Ab[i][m]) > tol:
            raise ValueError("Система несовместна")

    # Обратный ход Гаусса
    solutions = []
    free_vars = []
    lead_vars = [-1] * m

    # Определяем свободные переменные
    for i in range(rank):
        for j in range(m):
            if abs(Ab[i][j]) > tol:
                lead_vars[j] = i
                break

    for j in range(m):
        if lead_vars[j] == -1:
            free_vars.append(j)

    # Если свободных нет - единственное решение
    if not free_vars:
        solution = [[0.0] for _ in range(m)]
        for i in range(rank):
            for j in range(m):
                if abs(Ab[i][j]) > tol:
                    solution[j][0] = Ab[i][m]
                    break

        sol = [(i, 0, solution[i][0]) for i in range(m) if abs(solution[i][0]) > tol]
        return [Matrix(m, 1, sol)]

    # Если свободные есть - строим базис
    for free in free_vars:
        vec = [[0.0] for _ in range(m)]
        vec[free][0] = 1.0

        for i in range(rank):
            for j in range(m):
                if abs(Ab[i][j]) > tol:
                    sum_ax = 0.0
                    for k in range(j + 1, m):
                        sum_ax += Ab[i][k] * vec[k][0]
                    vec[j][0] = Ab[i][m] - sum_ax
                    break

        basis = [(i, 0, vec[i][0]) for i in range(m) if abs(vec[i][0]) > tol]
        solutions.append(Matrix(m, 1, basis))

    return solutions

In [384]:
# Пример системы уравнений:
# 1x + 2y = 5
# 3x + 4y = 11

A = save_matrix([[1, 2], [3, 4]])
b = save_matrix([[5], [11]])

solutions = gauss_solver(A, b)
if len(solutions) == 1:
    print("Единственное решение:")
    print(solutions[0])
else:
    print(f"Бесконечно много решений. Базис ФСР (размерность {len(solutions)}):")
    for i, sol in enumerate(solutions):
        print(f"Решение {i+1}:")
        print(sol)

Единственное решение:
1.0
2.0


# Easy 2: Центрирование данных
$$ X_{centered} = X - X_{mean} $$

In [385]:
def get_means(X: Matrix) -> List[float]:
    rows = X.rows
    cols = X.columns
    means = [0.0] * cols

    for col in range(1, cols + 1):
        for row in range(1, rows + 1):
            means[col-1] += X.get(row, col)
        means[col-1] /= rows

    return means

def center_data(X: Matrix) -> Matrix:
    means = get_means(X)
    centered_data = []

    for row in range(1, X.rows + 1):
        for col in range(1, X.columns + 1):
            val = X.get(row, col)
            centered_val = val - means[col-1]
            centered_data.append((row-1, col-1, centered_val))

    return Matrix(X.rows, X.columns, centered_data)

Сравнение с NumPy:

In [386]:
X = generate_matrix(3, 3, -3, 3)

print("Исходная матрица:")
print(X)

X_centered = center_data(X)
print("\nНаше центрирование:")
print(X_centered)

X_np = np.array(get_classic_format(X))
centered_np = X_np - X_np.mean(axis=0)

print("\nNumPy центрирование:")
print(centered_np)


Исходная матрица:
-3.0 -1.0 1.0
1.0 -3.0 -2.0
1.0 -1.0 0

Наше центрирование:
-2.6666666666666665 0.6666666666666667 1.3333333333333333
1.3333333333333333 -1.3333333333333333 -1.6666666666666667
1.3333333333333333 0.6666666666666667 0.3333333333333333

NumPy центрирование:
[[-2.66666667  0.66666667  1.33333333]
 [ 1.33333333 -1.33333333 -1.66666667]
 [ 1.33333333  0.66666667  0.33333333]]


# Easy 3: матрица ковариаций
$$ C = \frac{1}{n-1}X^TX $$

In [387]:
def covariance_matrix(X_centered: Matrix) -> Matrix:
    n = X_centered.rows
    p = X_centered.columns
    X = get_classic_format(X_centered)

    cov_data = []
    for i in range(p):
        for j in range(p):
            cov = sum(X[row][i] * X[row][j] for row in range(n)) / (n - 1)
            cov_data.append((i, j, cov))

    return Matrix(p, p, cov_data)

Сравнение с NumPy:

In [388]:
X = generate_matrix(5, 5, -10, 10)


X_centered = center_data(X)
cov = covariance_matrix(X_centered)

X_np = np.array(get_classic_format(X))
cov_numpy = np.cov(X_np, rowvar=False, bias=False)


print("Наш результат:")
print(cov)
print("\nNumPy результат:")
print(cov_numpy)


Наш результат:
29.700000000000003 -23.95 -21.400000000000002 9.9 7.1
-23.95 31.7 17.4 -11.9 -21.35
-21.400000000000002 17.4 49.8 10.45 -30.2
9.9 -11.9 10.45 14.3 -2.3000000000000007
7.1 -21.35 -30.2 -2.3000000000000007 53.3

NumPy результат:
[[ 29.7  -23.95 -21.4    9.9    7.1 ]
 [-23.95  31.7   17.4  -11.9  -21.35]
 [-21.4   17.4   49.8   10.45 -30.2 ]
 [  9.9  -11.9   10.45  14.3   -2.3 ]
 [  7.1  -21.35 -30.2   -2.3   53.3 ]]


# Normal 1: Собственные значения

In [389]:
# поиск корня бисекцией
def root_search(f, a, b, tol=1e-6):
    fa, fb = f(a), f(b)
    if fa * fb > 0:
        return None

    while (b-a) > tol:
        c = (a + b) / 2
        fc = f(c)
        if abs(fc) < tol:
            return c
        if fa * fc < 0:
            b, fb = c, fc
        else:
            a, fa = c, fc
    return (a + b) / 2

# поиск экстремума бисекцией
def extremum_search(f, a0, b0, epsilon, delta=1e-10):
    a = a0
    b = b0
    ans = (a + b) / 2

    while abs(b - a) > epsilon:
        yk = (a + b - delta) / 2
        zk = (a + b + delta) / 2

        if f(yk) <= f(zk):
            b = zk
        else:
            a = yk

        ans = (a + b) / 2
    return ans

In [390]:
# основная функция. поиск собственных значений
def find_eigenvalues(C_matrix: 'Matrix', tol: float = 1e-6) -> List[float]:
    n = C_matrix.rows
    C = get_classic_format(C_matrix)

# обычный характеристический полином
    def characteristic_poly(lam):
        mat = [[C[i][j] - (lam if i == j else 0) for j in range(n)] for i in range(n)]
        #return determinant(mat)
        # чтоб оперативней считалось, потом закомментить
        return np.linalg.det(mat)

    # модуль полинома для корней чётной кратности
    def poly_abs(lam):
        mat = [[C[i][j] - (lam if i == j else 0) for j in range(n)] for i in range(n)]
        #return abs(determinant(mat))
        # чтоб оперативней считалось, потом закомментить
        return abs(np.linalg.det(mat))

    # Вычисляем интервал поиска (см теорему Гершгорина)
    Gershgorin_intervals = []
    for i in range(n):
        radius = sum(abs(C[i][j]) for j in range(n) if j != i)
        center = C[i][i]
        Gershgorin_intervals.append((center - radius, center + radius))

    lower = min(interval[0] for interval in Gershgorin_intervals)
    upper = max(interval[1] for interval in Gershgorin_intervals)

    # Расширяем интервал, ну чтобы понадёжнее
    lower -= 3
    upper += 3
    count = max(100, 10 * n)

    # непосредственно поиск корней
    def find_roots(f, fabs, a, b, tol, count):
        roots = []
        # дробим [a,b] на мелкие подотрезки
        step = (b - a) / count

        x_prev = a
        f_prev = f(x_prev)

        # пробегаем по каждому маленькому отрезку
        for i in range(1, count+1):
            x = a + i * step
            fx = f(x)

            # если попали в корень
            if abs(fx) < tol:
                roots.append(x)
            # если функция меняет знак, ищем корень
            # если не меняет, смотрим экстремум для функции-модуля.
            if f_prev * fx < 0:
                roots.append(root_search(f, x_prev, x, tol))
            else:
                exst = extremum_search(fabs, x_prev, x, tol)
                # если при поиске экстремума «скатились» близко к нулю, то всё ок
                if abs(f(exst)) < 0.0001:
                  roots.append(exst)

            x_prev = x
            f_prev = fx

        # убираем дубликаты (близкие корни)
        unique_roots = [roots[0]]
        for i in range(1,len(sorted(roots))):
            if not unique_roots or (roots[i] - roots[i-1] > 2*step):
                unique_roots.append(roots[i])

        return unique_roots

    eigenvalues = find_roots(characteristic_poly, poly_abs, lower, upper, tol, count)
    return sorted(eigenvalues)[::-1]

In [391]:
C = generate_matrix(5, 5, -10, 10)
print("Исходная матрица:")
print(C)

eigenvalues = find_eigenvalues(C)
print("\nНайденные собственные значения:")
for i, val in enumerate(eigenvalues, 1):
    print(f"lambda{i} = {val:.6f}")

np_matrix = get_classic_format(C)
np_eigenvalues = np.linalg.eigvals(np_matrix)
print("\nПроверка с numpy:")
for i, val in enumerate(sorted(np_eigenvalues, reverse=True), 1):
    print(f"lambda{i} = {val:.6f}")

Исходная матрица:
-6.0 8.0 3.0 6.0 -5.0
8.0 3.0 1.0 8.0 6.0
6.0 -3.0 -3.0 -2.0 2.0
-4.0 3.0 4.0 -10.0 10.0
4.0 4.0 -10.0 2.0 -9.0

Найденные собственные значения:
lambda1 = 10.401912
lambda2 = 0.747499
lambda3 = -10.655391

Проверка с numpy:
lambda1 = 10.401912+0.000000j
lambda2 = 0.747500+0.000000j
lambda3 = -10.655391+0.000000j
lambda4 = -12.747010+9.007453j
lambda5 = -12.747010-9.007453j


# Normal 2: Собственные векторы

In [392]:
def normalize_vector(vect: Matrix) -> Matrix:
    vector = get_classic_format(vect)
    norm = math.sqrt(sum(x[0] ** 2 for x in vector))
    return save_matrix([[x[0] / norm] for x in vector])

def find_eigenvectors(C_matrix: Matrix, eigenvalues: List[float], tol = 1e-2) -> List[Matrix]:
    n = C_matrix.rows
    vectors_list = []

    C = get_classic_format(C_matrix)
    for lam in eigenvalues:
        C_lam = save_matrix([[C[i][j] - (lam if i == j else 0) for j in range(n)] for i in range(n)])
        vectors_lam = gauss_solver(C_lam, save_matrix([[0]]*n), tol)
        vectors_list += vectors_lam

    normalize_list = []
    for vectors in vectors_list:
        normalize_list.append(normalize_vector(vectors))
    return normalize_list

Сравнение с NumPy:

In [393]:
# C = save_matrix([[-3, 18, 9, 50, 80],
#                [7, -86, -25, -15, 94],
#                [-65, 6, -66, 18, 67],
#                [22, -1, -39, -41, 87],
#                [-85, -3, 26, 2, 22]])
C = generate_matrix(5, 5, -10, 10)

X_centered = center_data(C)
covar = covariance_matrix(X_centered)
vals = find_eigenvalues(covar)
vects = find_eigenvectors(covar, vals)

np_covar = np.cov(get_classic_format(X_centered), rowvar=False)
np_vals, np_vects = np.linalg.eig(np_covar)

sorted_indices = np.argsort(np_vals)[::-1]
np_vals_sorted = np_vals[sorted_indices]
np_vects_sorted = np_vects[:, sorted_indices]

# Сравнение
print("Собственные значения:")
print("Наши:", vals)
print("NumPy:", np_vals_sorted)

print("Наши:", np_vects_sorted)
print("NumPy:", vects)



Собственные значения:
Наши: [138.09548343598843, 63.807886901497845, 33.86025633513926, -6.854534663550534e-08]
NumPy: [1.38095483e+02 6.38078870e+01 3.38602562e+01 3.23637358e+00
 7.28368998e-15]
Наши: [[ 0.59159569  0.56560721  0.50260937  0.02524912 -0.27721713]
 [ 0.51001966 -0.12236608 -0.0814319   0.4300836   0.73027627]
 [ 0.30387666  0.26248016 -0.84371183  0.12255915 -0.33450374]
 [-0.52641097  0.51149013  0.04354628  0.67505221  0.06064302]
 [ 0.14298585 -0.57844364  0.16433729  0.58623746 -0.52371453]]
NumPy: [0.591595684214264
0.5100196599383526
0.30387665704541683
-0.526410976334177
0.14298585312269382, -0.5656213830031034
0.12216417964472917
-0.2624867394009321
-0.511502946596483
0.57845813293891, 0.5026093684164282
-0.0814319031393434
-0.8437118265966062
0.043546278070764693
0.16433728505225406, 0.27721712774947926
-0.7302762774538101
0.33450373574763675
-0.06064303265989237
0.5237145176823161]


# Normal 3: Доля объяснённой дисперсии

In [394]:
def explained_variance_ratio(eigenvalues: List[float], k: int) -> float:
    return sum(eigenvalues[:k]) / sum(eigenvalues)

Сравнение с NumPy:

In [395]:
# C = save_matrix([[-3, 18, 9, 50, 80],
#                [7, -86, -25, -15, 94],
#                [-65, 6, -66, 18, 67],
#                [22, -1, -39, -41, 87],
#                [-85, -3, 26, 2, 22]])

C = generate_matrix(5, 5, -10, 10)
X_centered = center_data(C)
covar = covariance_matrix(X_centered)
vals = find_eigenvalues(covar)


np_covar = np.cov(get_classic_format(X_centered), rowvar=False)
np_vals = np.linalg.eigvals(np_covar)
np_vals_sorted = sorted(np_vals, reverse=True)

for k in [1, 3, 5]:
    our_ratio = explained_variance_ratio(vals, k)

    np_ratio = sum(np_vals_sorted[:k]) / sum(np_vals_sorted)

    print(f"k={k}:")
    print(f"Наш результат: {our_ratio:.6f}")
    print(f"NumPy результат: {np_ratio:.6f}\n")

k=1:
Наш результат: 0.582459
NumPy результат: 0.582459

k=3:
Наш результат: 0.976795
NumPy результат: 0.976795

k=5:
Наш результат: 1.000000
NumPy результат: 1.000000



# Hard 1: алгоритм PCA

In [396]:
def project_data(X_matrix: Matrix, eigenvectors: List[Matrix], k: int) -> Tuple[Matrix, Matrix]:
    n = X_matrix.rows
    X = get_classic_format(X_matrix)
    vector_matrix = []

    for i in range(n):
        row = []
        for v in eigenvectors[:k]:
            row.append(v.get(i + 1, 1))
        vector_matrix.append(row)

    vector_matrix = save_matrix(vector_matrix)
    projected_X = multiply(X_matrix, vector_matrix)

    return projected_X, vector_matrix

def pca(X: Matrix, k: int) -> Tuple[Matrix, float]:
    # центрирование данных и матрица ковариаций
    centered_X = center_data(X)
    covar_X = covariance_matrix(centered_X)

    # поиск собственных значений и векторов
    eigenvals = find_eigenvalues(covar_X)
    eigenvects = find_eigenvectors(covar_X, eigenvals)

    # проекция данных на главные компоненты
    projected_X = project_data(centered_X, eigenvects, k)
    dispersion_res = explained_variance_ratio(eigenvals, k)

    return projected_X[0], dispersion_res

In [397]:
from sklearn.decomposition import PCA


# C = save_matrix([[-3, 18, 92, 50, 80], [67, -86, -25, -15, 94], [-65, 60, -66, 18, 67], [22, -1, -39, -41, 87], [-85, -3, 26, 2, 22]])
C = generate_matrix(5, 5, -100, 100)
my_res = pca(C, 3)[0]
print(f"Наш результат: {my_res}")

pca = PCA(n_components=3)
data_pca = pca.fit_transform(get_classic_format(C))
print(f"Сравнение с sklearn: {data_pca}")


Наш результат: 71.66987230431494 68.63588211683665 46.11866149638022
-71.1614263290861 -41.18459209315065 58.5382776422945
-52.024710127382384 -38.00796330721499 -14.503919466226339
-95.78444592636656 46.824076608463656 -55.06461430671578
147.3007100785201 -36.26740332493466 -35.08840536573259
Сравнение с sklearn: [[ -71.6698723   -68.63588212   46.11866152]
 [  71.16142633   41.18459209   58.53827763]
 [  52.02471013   38.00796331  -14.50391948]
 [  95.78444593  -46.82407661  -55.06461429]
 [-147.30071008   36.26740332  -35.08840538]]
